In [9]:
import pandas as pd
import os
from sqlalchemy import create_engine
import logging
import sqlite3
import time

In [2]:
DB_PATH = "inventory.db"
DATA_DIR = "data"
CHUNK_SIZE = 100_000

In [4]:
logging. basicConfig(
    filename="logs/ingestion_db.log" ,
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode = 'a'
)

In [5]:
engine = create_engine('sqlite:///inventory.db')

In [6]:
def ingest_csv_to_sqlite(file_path, table_name, conn):
    """Efficient ingestion using chunks + single transaction"""
    logging.info(f"Started ingesting {file_path} into {table_name}")
    start = time.time()
    
    with conn:  # Single transaction for the whole file
        for chunk in pd.read_csv(file_path, chunksize=CHUNK_SIZE):
            chunk.to_sql(table_name, conn, if_exists='append', index=False)
    
    end = time.time()
    logging.info(f"Finished {file_path} in {(end - start):.2f} seconds")

In [7]:
def load_raw_data():
    start_total = time.time()
    conn = sqlite3.connect(DB_PATH)
    
    for file in os.listdir(DATA_DIR):
        if file.endswith(".csv"):
            table_name = file[:-4]
            ingest_csv_to_sqlite(os.path.join(DATA_DIR, file), table_name, conn)
    
    conn.close()
    total_time = (time.time() - start_total) / 60
    logging.info("--------------Ingestion Complete-------------")
    logging.info(f"Total Time Taken: {total_time:.2f} minutes")


In [10]:
load_raw_data()